<a href="https://colab.research.google.com/github/aminehd/AminehTechnicalWritings/blob/main/SamplingStatsAndDistros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Outline
- Introduction (Purpose + scope)
- Body
  - sample summary statistics and examples (coefficients of fitted model)
  - load data and showcase
  - Sampling distribution of summary statistics. (use sample mean as example)
  - Expected value of summary statistics
  - Variance of summary statistic
  - Variance within a sample vs variance of a given stat over samplings
  - Standrad Error of stats and confidence interval. (SE(beta0))
  - F-test (within groups variance vs between group variance)
- Conclusion:

### Summary Statistic

Previously we explained the concept of population and the concept of sample. Basically we have one population (e.g., population of American people) but we can sample a subset in many ways (e.g., sample of America people under age 18, sample of American people living in new your, etc).

In fact if N is population size, there is $2^N$ samples.

Now a statistic or measure can be defined on the whole population as well as any given sample. A statistic or a measure in mathematic is a generalization of concepts such as lenght or heigh for a given set. Since samples and the population are sets, we can define such things on them. A well known one is mean. Thus you can think of population mean or any given sample mean.

So far it was fairly easy to explain things. But it gets a bit trickier when we wanna talk the distribution of such measures over allll samples and compare them with the value of measure on the poulation.

The motivation behind sampling is that population is too big and it is virtually impossible to calculate the mean for the whole population. Let's say as a minister of something you need the average age of your population.  HERE!!!



https://en.wikipedia.org/wiki/Sampling_distribution
start with mu but get to beta_i HERE!!!
The concept of unbiased estimator pops up one more time. HERE!!!


Then we look at expected value of sample statistices over multiple sampling. Also we look at variance of stuff within sample, and variance of sample stat over several samplings. This indeed is an intersting case when sample stat is defined to be the mean of sample. HERE!!!




notebook we try to look at the model coefficients ( 𝛽𝑖 's ) as summary statistics defined on samples. You can also think of them as measures since samples are subsets.



These can help understand concept of Standard Error and F-test better.